In [6]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import FinanceDataReader as fdr
import json

In [10]:

def zipyo(request):
    def main(code):
        res = requests.get(BASE_URL + str(code) + '&page=' + str(START_PAGE))
        page_soup = BeautifulSoup(res.text, 'lxml')

        total_page_num = page_soup.select_one('td.pgRR > a')
        total_page_num = int(total_page_num.get('href').split('=')[-1])

        ipt_html = page_soup.select_one('div.subcnt_sise_item_top')
        global fields
        fields = [item.get('value') for item in ipt_html.select('input')]

        result = [crawl(code,str(page)) for page in range(1, total_page_num + 1)]

        df = pd.concat(result, axis = 0, ignore_index = True)
        return df

    def crawl(code, page):
        global fields
        data = {'menu': 'market_sum',
                'fieldIds': fields,
                'returnUrl': BASE_URL + str(code) + "&page=" + str(page)}

        res = requests.post('https://finance.naver.com/sise/field_submit.nhn', data = data)
        page_soup = BeautifulSoup(res.text, 'lxml')

        table_html = page_soup.select_one('div.box_type_l')

        header_data = [item.get_text().strip() for item in table_html.select('thead th')][1:-1]

        inner_data = [item.get_text().strip() for item in table_html.find_all(lambda x:
                                                                                (x.name == 'a' and
                                                                                 'tltle' in x.get('class', [])) or
                                                                                (x.name == 'td' and
                                                                                 'number' in x.get('class', []))
                                                                                )]
        no_data = [item.get_text().strip() for item in table_html.select('td.no')]
        number_data = np.array(inner_data)

        number_data.resize(len(no_data), len(header_data))

        df = pd.DataFrame(data = number_data, columns = header_data)
        return df

    
    BASE_URL = 'https://finance.naver.com/sise/sise_market_sum.nhn?sosok='
    KOSPI_CODE = 0
    KOSDAK_CODE = 1
    START_PAGE = 1
    field = []
    KOSPI = main(KOSPI_CODE)
    KOSDAK = main(KOSDAK_CODE)
    KOSPI = KOSPI.replace('N/A',np.nan).dropna(axis = 0)
    KOSPI['PER'] = KOSPI['PER'].str.replace(',','').astype('float64')
    KOSPI['ROE'] = KOSPI['ROE'].str.replace(',','').astype('float64')
    KOSPI['ROA'] = KOSPI['ROA'].str.replace(',','').astype('float64')
    KOSPI['시가총액'] = KOSPI['시가총액'].str.replace(',','').astype('int64')
    KOSPI['매출액'] = KOSPI['매출액'].str.replace(',','').astype('int64')
    KOSPI['영업이익'] = KOSPI['영업이익'].str.replace(',','').astype('int64')
    KOSPI['PSR'] = KOSPI['시가총액'] / KOSPI['매출액']
    KOSPI['POR'] = KOSPI['시가총액'] / KOSPI['영업이익']
    KOSPI = KOSPI[['종목명','PER','ROE','ROA','PBR','PSR','POR']]
    c = ['PER','POR']
    KOSPI[c] = KOSPI[c].mask(KOSPI[c] < 0)
    KOSPI = KOSPI.dropna(axis = 0)
    KOSPI['PBR + PSR'] = KOSPI[['PBR','PSR']].rank(method = 'min').sum(axis = 1)
    KOSPI['PBR + PSR + POR'] = KOSPI[['PBR','PSR','POR']].rank(method = 'min').sum(axis = 1)
    KOSPI['total_rank'] = (KOSPI[['PER','PBR','PSR']].rank(method = 'min').sum(axis = 1) + KOSPI[['ROE','ROA']].rank(method = 'min',ascending = False).sum(axis = 1))
    PBR_PSR = KOSPI[['종목명','PBR','PSR','PBR + PSR']].sort_values(by = 'PBR + PSR')
    PBR_PSR = PBR_PSR.drop(['PBR + PSR'],axis = 1)
    PBR_PSR.index = list(range(1,len(PBR_PSR)+1))
    df = PBR_PSR.copy()
    json_records = df.reset_index().to_json(orient ='records') 
    data = [] 
    data = json.loads(json_records) 
    context = {'d': data} 

    return render(request, 'selnm/pbr_psr.html', context) 
 

KeyboardInterrupt: 